# The Multiparameter Intelligent Monitoring in Intensive Care II (MIMIC-II)

Goldberger, Ary L., et al. "PhysioBank, PhysioToolkit, and PhysioNet: components of a new research resource for complex physiologic signals." circulation 101.23 (2000): e215-e220.

Saeed, Mohammed, et al. "Multiparameter Intelligent Monitoring in Intensive Care II (MIMIC-II): a public-access intensive care unit database." Critical care medicine 39.5 (2011): 952.

## Import libraries

In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

from torch_explain.models.explainer import MuExplainer
from torch_explain.logic.metrics import formula_consistency
from experiments.data.load_datasets import load_mimic

## Import MIMIC-II dataset

In [2]:
x, y, concept_names = load_mimic()

dataset = TensorDataset(x, y)

train_size = int(len(dataset) * 0.5)
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_data, batch_size=train_size)
val_loader = DataLoader(val_data, batch_size=val_size)
test_loader = DataLoader(test_data, batch_size=test_size)

n_concepts = next(iter(train_loader))[0].shape[1]
n_classes = 2

print(concept_names)
print(n_concepts)
print(n_classes)

['aline_flg', 'gender_num', 'sepsis_flg', 'chf_flg', 'afib_flg', 'renal_flg', 'liver_flg', 'copd_flg', 'cad_flg', 'stroke_flg', 'mal_flg', 'resp_flg', 'abg_count_LOW', 'abg_count_NORMAL', 'abg_count_HIGH', 'age_LOW', 'age_NORMAL', 'age_HIGH', 'bmi_LOW', 'bmi_NORMAL', 'bmi_HIGH', 'bun_first_LOW', 'bun_first_NORMAL', 'bun_first_HIGH', 'chloride_first_LOW', 'chloride_first_NORMAL', 'chloride_first_HIGH', 'creatinine_first_LOW', 'creatinine_first_NORMAL', 'creatinine_first_HIGH', 'day_icu_intime_num_LOW', 'day_icu_intime_num_NORMAL', 'day_icu_intime_num_HIGH', 'hgb_first_LOW', 'hgb_first_NORMAL', 'hgb_first_HIGH', 'hour_icu_intime_LOW', 'hour_icu_intime_NORMAL', 'hour_icu_intime_HIGH', 'hr_1st_LOW', 'hr_1st_NORMAL', 'hr_1st_HIGH', 'icu_los_day_LOW', 'icu_los_day_NORMAL', 'icu_los_day_HIGH', 'iv_day_1_LOW', 'iv_day_1_NORMAL', 'iv_day_1_HIGH', 'map_1st_LOW', 'map_1st_NORMAL', 'map_1st_HIGH', 'pco2_first_LOW', 'pco2_first_NORMAL', 'pco2_first_HIGH', 'platelet_first_LOW', 'platelet_first_NORMA

## 10-fold cross-validation with explainer network

In [3]:
seed_everything(42)

base_dir = f'./results/mimic-ii/explainer'
os.makedirs(base_dir, exist_ok=True)

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
results_list = []
explanations = {i: [] for i in range(n_classes)}
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), 
                                                               y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
    y_trainval, y_test = torch.FloatTensor(y[trainval_index]), torch.FloatTensor(y[test_index])
    x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.2, random_state=42)
    print(f'{len(y_train)}/{len(y_val)}/{len(y_test)}')

    train_data = TensorDataset(x_train, y_train)
    val_data = TensorDataset(x_val, y_val)
    test_data = TensorDataset(x_test, y_test)
    train_loader = DataLoader(train_data, batch_size=train_size)
    val_loader = DataLoader(val_data, batch_size=val_size)
    test_loader = DataLoader(test_data, batch_size=test_size)

    checkpoint_callback = ModelCheckpoint(dirpath=base_dir, monitor='val_loss', save_top_k=1)
    trainer = Trainer(min_epochs=200, max_epochs=200, gpus=1, auto_lr_find=True, deterministic=True,
                      check_val_every_n_epoch=1, default_root_dir=base_dir,
                      weights_save_path=base_dir, callbacks=[checkpoint_callback])
    model = MuExplainer(n_concepts=n_concepts, n_classes=n_classes, l1=0.001, lr=0.01, 
                        explainer_hidden=[20], awareness='l1')

    start = time.time()
    trainer.fit(model, train_loader, val_loader)
    model.freeze()
    model_results = trainer.test(model, test_dataloaders=test_loader)
    results, f = model.explain_class(val_loader, test_loader,
                                     topk_explanations=1000,
                                     max_minterm_complexity=10,
                                     concept_names=concept_names)
    end = time.time() - start
    results['model_accuracy'] = model_results[0]['test_acc']
    results['extraction_time'] = end
    # print(f"Gamma: {model.model[0].gamma}")

    results_list.append(results)
    for j in range(n_classes):
        # print(f"Explanation: {f[j]['explanation']}")
        print(f"Explanation accuracy: {f[j]['explanation_accuracy']}")
        explanations[j].append(f[j]['explanation'])

consistencies = []
for j in range(n_classes):
    consistencies.append(formula_consistency(explanations[j]))
explanation_consistency = np.mean(consistencies)
results['explanation_consistency'] = explanation_consistency

Global seed set to 42


Split [1/10]
1278/320/178


c:\users\pietr\anaconda3\envs\deep-logic\lib\site-packages\pytorch_lightning\utilities\distributed.py:52: UserWarning: Checkpoint directory ./results/mimic-ii/explainer exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | loss  | NLLLoss    | 0     
1 | model | Sequential | 3.7 K 
-------------------------------------
3.7 K     Trainable params
0         Non-trainable params
3.7 K     Total params
0.015     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.790960431098938, 'test_acc_epoch': 0.790960431098938}
--------------------------------------------------------------------------------
Explanation accuracy: 0.7288135593220338
Explanation accuracy: 0.7909604519774012


## Print results

In [4]:
f[j]['explanation']

'(chf_flg & resp_flg & age_HIGH & ~stroke_flg & ~mal_flg & ~age_LOW & ~hgb_first_HIGH & ~service_num_HIGH) | (stroke_flg & age_HIGH & service_num_HIGH & ~chf_flg & ~mal_flg & ~resp_flg & ~age_LOW & ~sapsi_first_LOW) | (chf_flg & afib_flg & stroke_flg & resp_flg & age_HIGH & ~mal_flg & ~age_LOW & ~hgb_first_HIGH & ~sapsi_first_LOW) | (resp_flg & age_HIGH & ~stroke_flg & ~mal_flg & ~age_LOW & ~hgb_first_HIGH & ~sapsi_first_LOW & ~service_num_HIGH) | (stroke_flg & age_HIGH & hgb_first_HIGH & service_num_HIGH & ~chf_flg & ~afib_flg & ~resp_flg & ~age_LOW & ~sapsi_first_LOW) | (chf_flg & afib_flg & resp_flg & ~stroke_flg & ~mal_flg & ~age_LOW & ~hgb_first_HIGH & ~sapsi_first_LOW & ~service_num_HIGH) | (chf_flg & afib_flg & age_HIGH & ~stroke_flg & ~mal_flg & ~age_LOW & ~hgb_first_HIGH & ~sapsi_first_LOW & ~service_num_HIGH) | (afib_flg & stroke_flg & age_HIGH & ~chf_flg & ~mal_flg & ~resp_flg & ~age_LOW & ~hgb_first_HIGH & ~sapsi_first_LOW) | (afib_flg & resp_flg & service_num_HIGH & ~chf_f

In [5]:
results_df = pd.DataFrame(results_list)
results_df

,explanation_accuracy,explanation_fidelity,explanation_complexity,model_accuracy,extraction_time,explanation_consistency
0,0.727528,0.817416,183.0,0.803371,41.359339,NaN
1,0.747191,0.769663,176.5,0.786517,41.079742,NaN
2,0.707865,0.786517,209.0,0.792135,42.362944,NaN
3,0.764045,0.825843,158.0,0.758427,35.836830,NaN
4,0.741573,0.825843,151.5,0.792135,39.652550,NaN
5,0.750000,0.806180,203.5,0.803371,48.891810,NaN
6,0.717514,0.813559,198.5,0.779661,41.221272,NaN
7,0.765537,0.822034,151.0,0.841808,38.283163,NaN
8,0.745763,0.853107,236.0,0.802260,42.634752,NaN
9,0.759887,0.827684,227.0,0.790960,44.695469,0.488095


In [6]:
results_df.mean()

explanation_accuracy         0.742690
explanation_fidelity         0.814784
explanation_complexity     189.400000
model_accuracy               0.795064
extraction_time             41.601787
explanation_consistency      0.488095
dtype: float64

## Compare with out-of-the-box models

In [7]:
dt_scores, rf_scores = [], []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = x[trainval_index], x[test_index]
    y_trainval, y_test = y[trainval_index].argmax(dim=1), y[test_index].argmax(dim=1)

    dt_model = DecisionTreeClassifier(max_depth=9, random_state=split)
    dt_model.fit(x_trainval, y_trainval)
    dt_scores.append(dt_model.score(x_test, y_test))

    rf_model = RandomForestClassifier(random_state=split)
    rf_model.fit(x_trainval, y_trainval)
    rf_scores.append(rf_model.score(x_test, y_test))

print(f'Random forest scores: {np.mean(rf_scores)} (+/- {np.std(rf_scores)})')
print(f'Decision tree scores: {np.mean(dt_scores)} (+/- {np.std(dt_scores)})')
print(f'Mu net scores (model): {results_df["model_accuracy"].mean()} (+/- {results_df["model_accuracy"].std()})')
print(f'Mu net scores (exp): {results_df["explanation_accuracy"].mean()} (+/- {results_df["explanation_accuracy"].std()})')

Split [1/10]
Split [2/10]
Split [3/10]
Split [4/10]
Split [5/10]
Split [6/10]
Split [7/10]
Split [8/10]
Split [9/10]
Split [10/10]
Random forest scores: 0.7928077191646036 (+/- 0.01965065471826671)
Decision tree scores: 0.753364438519647 (+/- 0.018974185154691948)
Mu net scores (model): 0.7950644195079803 (+/- 0.021242922705987138)
Mu net scores (exp): 0.7426902812162762 (+/- 0.019535167387036503)
